<a href="https://colab.research.google.com/github/Bray-Nyagwoka/ISTG6010-2025/blob/main/Week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# Game Classes


import random

class ShotResult:
    HIT = "Hit"
    MISS = "Miss"
    SUNK = "Sunk"
    INVALID = "Invalid"
    ALREADY_TAKEN = "Already Taken"

class Ship:
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.positions = []
        self.hits = set()

    def place(self, start_row, start_col, horizontal):
        self.positions.clear()
        for i in range(self.size):
            row = start_row
            col = start_col
            if horizontal:
                col += i
            else:
                row += i
            self.positions.append((row, col))

    def register_hit(self, position):
        if position in self.positions:
            self.hits.add(position)
            return True
        return False

    def is_sunk(self):
        return set(self.positions) == self.hits

class Grid:
    def __init__(self, size=10):
        self.size = size
        self.ships = []
        self.shots = set()

    def is_valid_position(self, ship):
        for pos in ship.positions:
            r, c = pos
            if not (0 <= r < self.size and 0 <= c < self.size):
                return False
            for existing in self.ships:
                if pos in existing.positions:
                    return False
        return True

    def add_ship(self, ship):
        if self.is_valid_position(ship):
            self.ships.append(ship)
            return True
        return False

    def receive_shot(self, position):
        if position in self.shots:
            return ShotResult.ALREADY_TAKEN
        self.shots.add(position)
        for ship in self.ships:
            if ship.register_hit(position):
                if ship.is_sunk():
                    return ShotResult.SUNK
                return ShotResult.HIT
        return ShotResult.MISS

    def all_ships_sunk(self):
        return all(ship.is_sunk() for ship in self.ships)

    def display_own_grid(self):
        grid = [['~' for _ in range(self.size)] for _ in range(self.size)]
        for ship in self.ships:
            for r, c in ship.positions:
                grid[r][c] = 'S'
        for r, c in self.shots:
            if grid[r][c] == 'S':
                grid[r][c] = 'X'
            else:
                grid[r][c] = 'O'
        print("Brian's Grid:")
        self.print_grid(grid)

    def display_opponent_grid(self):
        grid = [['~' for _ in range(self.size)] for _ in range(self.size)]
        for r, c in self.shots:
            grid[r][c] = 'X'
        print("Opponent Grid (Brian's shots):")
        self.print_grid(grid)

    def print_grid(self, grid):
        print("  " + " ".join(str(i) for i in range(self.size)))
        for idx, row in enumerate(grid):
            print(f"{idx} " + " ".join(row))

class Player:
    def __init__(self, name):
        self.name = name
        self.grid = Grid()

    def place_ships_randomly(self, ship_definitions):
        for name, size in ship_definitions:
            placed = False
            while not placed:
                horizontal = random.choice([True, False])
                row = random.randint(0, self.grid.size - 1)
                col = random.randint(0, self.grid.size - 1)
                ship = Ship(name, size)
                ship.place(row, col, horizontal)
                placed = self.grid.add_ship(ship)

    def take_turn(self, opponent_grid):
        while True:
            try:
                user_input = input(f"{self.name}, enter shot (row,col): ")
                row, col = map(int, user_input.strip().split(','))
                if not (0 <= row < opponent_grid.size and 0 <= col < opponent_grid.size):
                    raise ValueError
                result = opponent_grid.receive_shot((row, col))
                print(f"{self.name}'s shot at ({row},{col}): {result}")
                return result
            except ValueError:
                print("Invalid input. Enter coordinates like 2,3.")

class Game:
    def __init__(self):
        self.ship_definitions = [("Destroyer", 2), ("Submarine", 3), ("Battleship", 4)]
        self.brian = Player("Brian")
        self.computer = Player("Computer")

    def setup(self):
        print("Placing Brian's ships randomly...")
        self.brian.place_ships_randomly(self.ship_definitions)
        print("Placing computer ships randomly...")
        self.computer.place_ships_randomly(self.ship_definitions)

    def play(self):
        print("\n=== New Game Starting ===\n")
        self.setup()
        turn = 0
        while True:
            if turn % 2 == 0:
                result = self.brian.take_turn(self.computer.grid)
            else:
                while True:
                    row = random.randint(0, self.computer.grid.size - 1)
                    col = random.randint(0, self.computer.grid.size - 1)
                    if (row, col) not in self.brian.grid.shots:
                        break
                result = self.brian.grid.receive_shot((row, col))
                print(f"Computer's shot at ({row},{col}): {result}")

            if self.computer.grid.all_ships_sunk():
                print("\n Brian wins!\n")
                break
            elif self.brian.grid.all_ships_sunk():
                print("\n Computer wins!\n")
                break
            turn += 1

# Unit Tests


import unittest

# ---- Test Ship Class ----
class TestShip(unittest.TestCase):
    def test_ship_initialization(self):
        ship = Ship("Cruiser", 3)
        self.assertEqual(ship.name, "Cruiser")
        self.assertEqual(ship.size, 3)

    def test_place_ship_horizontal(self):
        ship = Ship("Destroyer", 2)
        ship.place(0, 0, True)
        self.assertEqual(ship.positions, [(0, 0), (0, 1)])

    def test_register_hit(self):
        ship = Ship("Submarine", 3)
        ship.place(1, 1, False)
        hit = ship.register_hit((1, 1))
        self.assertTrue(hit)

    def test_is_sunk(self):
        ship = Ship("Mini", 1)
        ship.place(0, 0, True)
        ship.register_hit((0, 0))
        self.assertTrue(ship.is_sunk())

# ---- Test Grid Class ----
class TestGrid(unittest.TestCase):
    def test_add_ship_valid(self):
        grid = Grid()
        ship = Ship("Scout", 2)
        ship.place(0, 0, True)
        self.assertTrue(grid.add_ship(ship))

    def test_add_ship_overlap(self):
        grid = Grid()
        s1 = Ship("A", 2)
        s1.place(0, 0, True)
        s2 = Ship("B", 2)
        s2.place(0, 1, True)
        grid.add_ship(s1)
        self.assertFalse(grid.add_ship(s2))

    def test_receive_hit(self):
        grid = Grid()
        s = Ship("A", 1)
        s.place(1, 1, True)
        grid.add_ship(s)
        self.assertEqual(grid.receive_shot((1, 1)), ShotResult.SUNK)

    def test_receive_miss(self):
        grid = Grid()
        self.assertEqual(grid.receive_shot((5, 5)), ShotResult.MISS)

# ---- Test Player Class ----
class TestPlayer(unittest.TestCase):
    def test_player_name(self):
        p = Player("Alex")
        self.assertEqual(p.name, "Alex")

    def test_player_grid_exists(self):
        p = Player("A")
        self.assertIsInstance(p.grid, Grid)

    def test_random_placement_adds_ships(self):
        p = Player("B")
        p.place_ships_randomly([("X", 2), ("Y", 2)])
        self.assertEqual(len(p.grid.ships), 2)

    def test_receive_hit_on_player_grid(self):
        p = Player("P")
        s = Ship("Tiny", 1)
        s.place(0, 0, True)
        p.grid.add_ship(s)
        result = p.grid.receive_shot((0, 0))
        self.assertEqual(result, ShotResult.SUNK)

# ---- Test Game Class ----
class TestGame(unittest.TestCase):
    def test_game_creates_players(self):
        game = Game()
        self.assertIsInstance(game.brian, Player)
        self.assertIsInstance(game.computer, Player)

    def test_ship_definitions_length(self):
        game = Game()
        self.assertEqual(len(game.ship_definitions), 3)

    def test_setup_places_ships(self):
        game = Game()
        game.setup()
        self.assertEqual(len(game.brian.grid.ships), 3)

    def test_players_named_correctly(self):
        game = Game()
        self.assertEqual(game.brian.name, "Brian")
        self.assertEqual(game.computer.name, "Computer")

# ---- Test ShotResult Class ----
class TestShotResult(unittest.TestCase):
    def test_hit_constant(self):
        self.assertEqual(ShotResult.HIT, "Hit")

    def test_miss_constant(self):
        self.assertEqual(ShotResult.MISS, "Miss")

    def test_sunk_constant(self):
        self.assertEqual(ShotResult.SUNK, "Sunk")

    def test_unique_constants(self):
        values = {ShotResult.HIT, ShotResult.MISS, ShotResult.SUNK,
                  ShotResult.ALREADY_TAKEN, ShotResult.INVALID}
        self.assertEqual(len(values), 5)

# ---- Run All Tests ----
def run_tests():
    test_classes = [TestShip, TestGrid, TestPlayer, TestGame, TestShotResult]
    suite = unittest.TestSuite()
    for test_class in test_classes:
        tests = unittest.TestLoader().loadTestsFromTestCase(test_class)
        suite.addTests(tests)
    unittest.TextTestRunner().run(suite)

run_tests()


....................
----------------------------------------------------------------------
Ran 20 tests in 0.022s

OK


Placing Brian's ships randomly...
Placing computer ships randomly...
